In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from parquet_data_reader import ParquetDataReader
from models.user_based_simple import CollaborativeRecommender

import polars as pl
import numpy as np

## Data import and EDA

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
train_behaviors_df = pl.concat([train_behaviors_df, test_behaviours_df])

### Table contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove non-needed values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
150528,null,25.0,null,143471,false,null,null,null,false
153068,9778682,78.0,100.0,151570,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for missing values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(477534, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,335811,0,337955,0,0,445004,468353,465013,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(141723, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,3640,0,0,132011,138521,136899,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(141723, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,3640,0,0,0


We still see that 1780/70421 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for multiple instances of the same article and user

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (19_735, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9788705    ┆ 1790492 ┆ 2     │
│ 9789502    ┆ 512125  ┆ 2     │
│ 9782884    ┆ 429788  ┆ 3     │
│ 9772227    ┆ 1902391 ┆ 3     │
│ 9784856    ┆ 1021793 ┆ 3     │
│ …          ┆ …       ┆ …     │
│ 9771919    ┆ 2582723 ┆ 2     │
│ 9784879    ┆ 460020  ┆ 2     │
│ 9784839    ┆ 1977921 ┆ 3     │
│ 9780439    ┆ 1746976 ┆ 2     │
│ 9775752    ┆ 1774664 ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\chris\AppData\Local\Temp\ipykernel_17816\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
train_behaviors_df = (
    train_behaviors_df
    .group_by(["article_id", "user_id"])
    .agg(
        pl.col("read_time").product().alias("total_readtime"),  # Multiply all readtime values
        pl.col("scroll_percentage").max().alias("max_scroll")  # Select the largest scroll percentage
    )
)

print(train_behaviors_df)

shape: (106_810, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9773617    ┆ 366611  ┆ 57.0           ┆ 100.0      │
│ 9777292    ┆ 1662583 ┆ 80.0           ┆ 100.0      │
│ 9783334    ┆ 413217  ┆ 78588.0        ┆ 100.0      │
│ 9774074    ┆ 686661  ┆ 62.0           ┆ 100.0      │
│ 9777200    ┆ 1577912 ┆ 63.0           ┆ 100.0      │
│ …          ┆ …       ┆ …              ┆ …          │
│ 9779648    ┆ 820522  ┆ 106.0          ┆ 100.0      │
│ 9790066    ┆ 1423532 ┆ 122.0          ┆ 100.0      │
│ 9788043    ┆ 1100577 ┆ 37.0           ┆ 100.0      │
│ 9771764    ┆ 2360333 ┆ 128.0          ┆ 100.0      │
│ 9774352    ┆ 134499  ┆ 45.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

In [18]:
recommender = CollaborativeRecommender(train_behaviors_df)
recommender.fit()

{366611: [(802025, np.float64(0.2357022603955159)),
  (473054, np.float64(0.20573779994945585)),
  (891292, np.float64(0.20412414523193156)),
  (79433, np.float64(0.19245008972987532)),
  (1419406, np.float64(0.19245008972987532)),
  (547571, np.float64(0.19245008972987532)),
  (2035340, np.float64(0.19245008972987532)),
  (1176137, np.float64(0.19245008972987532)),
  (1806965, np.float64(0.19245008972987532)),
  (2295954, np.float64(0.19245008972987532))],
 1662583: [(888883, np.float64(0.40824829046386313)),
  (1734983, np.float64(0.40824829046386313)),
  (1506696, np.float64(0.40824829046386313)),
  (1783421, np.float64(0.40824829046386313)),
  (1838608, np.float64(0.40824829046386313)),
  (1961096, np.float64(0.40824829046386313)),
  (1523308, np.float64(0.40824829046386313)),
  (2137630, np.float64(0.40824829046386313)),
  (2242313, np.float64(0.40824829046386313)),
  (2352014, np.float64(0.28867513459481287))],
 413217: [(1158140, np.float64(1.0)),
  (788910, np.float64(1.0)),
  

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model presentation

In [19]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5))

reccomended for user  630220 :  [9774074, 9771113, 9773137, 9784275, 9777705]
reccomended for user  620796 :  [9775927, 9784852, 9345280, 9775673]
reccomended for user  1067393 :  [9778731, 9098807, 9772768, 9782745, 9776916]
reccomended for user  1726258 :  [9774392, 9778942, 9771113, 9775905, 9786359]
reccomended for user  17205 :  [9771333, 9773464]


In [20]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200)
print("Results")
results

Results


{'MAP@K': np.float64(0.0), 'NDCG@K': np.float64(0.0)}

In [21]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read=True)
print("Results")
results

Results


{'MAP@K': np.float64(0.025747126436781613),
 'NDCG@K': np.float64(0.3221834891323957)}

## Model Experimentation

In [22]:
test_user_id = 630220 

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, None, 9782092, 9780815, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9784275, 9771113, 9773137, 9777705, 9782884, 9774074, 9777296, 9788149, 9789922, 9790752, 9784559, 9772000, 9783213, 9774079, 9787332, 9784856, 9788116, 9789418, 9773846, 9788524, 9782635, 9770450, 9781906, 9777492, 9780195, 9756785, 9783852, 9775500, 9767830, 9773350, 9784875, 9780096, 9778623, 9785829, 9759476, 9775568, 9765759, 9778971, 9785205, 9785049, 9785828, 9781998, 9704504, 9705657, 9769624, 9778021, 9772227, 9776303, 9774096, 9785350, 9786204, 9775489, 9779807, 9773873, 9781947, 9789911, 9787264, 9784097, 9776369, 9785111, 9772543, 9784939, 9789810, 9774297, 9778074, 9770051, 9769348, 9776420, 97

In [23]:
test_user_id = 630220 

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, None, 9782092, 9780815, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9769504, 9759955, 9771627, 9775484, 9770045, 9771113, 9782884, 9779777, 9778657, 9784275, 9784591, 9778939, 9774142, 9771916, 9774120, 9788362, 9773137, 9778351, 9771948, 9774074, 9777705, 9787510, 9784856, 9788524, 9780096, 9775985, 9777492, 9787465, 9770450, 9789473, 9777296, 9777856, 9780195, 9756785, 9786111, 9788149, 9769531, 9787332, 9783824, 9783852, 9789922, 9758424, 9775500, 9772000, 9784875, 9782635, 9774079, 9775518, 9790752, 9767830, 9785829, 9781906, 9783213, 9778623, 9773846, 9789418, 9776691, 9788116, 9773350, 9787524, 9784559, 9770452, 9782919, 9773297, 9756075, 9693002, 9783404, 9787525, 97